# E14 폐렴 진단


## 들어가기 전에 

데이터 출저: 캐글 Chest X-Ray images 

[Chest X-Ray Images (Pneumonia)](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia)

``` terminal
$ mkdir -p ~/aiffel/chest_xray
$ cd ~/Downloads && unzip archive.zip -d ~/aiffel
```

### 사전 정보

의료 영상 촬영의 세 단면

* Sagittal plane : 시상면. 사람을 왼쪽과 오른쪽을 나누는 면.

* Coronal plane : 관상면. 인체를 앞 뒤로 나누는 면.

* Transverse plane : 횡단면(수평면). 인체를 상하로 나누는 면.

[폐렴](https://ko.wikipedia.org/wiki/%ED%8F%90%EB%A0%B4)

[염증](https://ko.wikipedia.org/wiki/%EC%97%BC%EC%A6%9D)

## Set-up

### library

In [2]:
import re    # 정규표현식 관련된 작업에 필요한 패키지
import os    # I/O 관련된 작업에 필요한 패키지 
import pandas as pd     # 데이터 전처리 관련된 작업에 필요한 패키지
import numpy as np      # 데이터 array 작업에 필요한 패키지
import tensorflow as tf  # 딥러닝 관련된 작업에 필요한 패키지
import matplotlib.pyplot as plt    # 데이터 시각화에 관련된 작업에 필요한 패키지
from sklearn.model_selection import train_test_split  # 데이터 전처리에 필요한 패키지

In [3]:
# 데이터 로드할 때 빠르게 로드할 수 있도록하는 설정 변수
AUTOTUNE = tf.data.experimental.AUTOTUNE

# 데이터 ROOT 경로 변수
ROOT_PATH = os.path.join(os.getenv('HOME'), 'aiffel')

# BATCH_SIZE 변수
BATCH_SIZE = 16

# X-RAY 이미지 사이즈 변수
IMAGE_SIZE = [180, 180]

# EPOCH 크기 변수
EPOCHS = 25

print(ROOT_PATH)

/home/ssac4/aiffel


### load data

In [4]:
train_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/train/*/*'))
test_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/test/*/*'))
val_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/val/*/*'))

print(len(train_filenames))
print(len(test_filenames))
print(len(val_filenames))

5216
624
16


원 validation 데이터가 너무 적어서 train에서 일부 뽑아서 분할합니다.

In [5]:
filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/train/*/*'))
filenames.extend(tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/val/*/*')))

# train, test(val) dataset으로 분할. test_size에 0.2는 20%롤 의미함.
train_filenames, val_filenames = train_test_split(filenames, test_size=0.2)

print(len(train_filenames))
print(len(val_filenames))

4185
1047


train 데이터 안의 정상 이미지와 폐렴 이미지 수 체크 

In [6]:
COUNT_NORMAL = len([filename for filename in train_filenames if "NORMAL" in filename])
print("Normal images count in training set: " + str(COUNT_NORMAL))

COUNT_PNEUMONIA = len([filename for filename in train_filenames if "PNEUMONIA" in filename])
print("Pneumonia images count in training set: " + str(COUNT_PNEUMONIA))

Normal images count in training set: 1059
Pneumonia images count in training set: 3126


정상보다 폐렴 이미지가 3배 많음 

In [7]:
train_list_ds  = tf.data.Dataset.from_tensor_slices(train_filenames)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_filenames)

 tf.data을 통해 배치 처리 작업을 효율적으로 할 수 있음. 

In [8]:
TRAIN_IMG_COUNT = tf.data.experimental.cardinality(train_list_ds).numpy()

print("Training images count: " + str(TRAIN_IMG_COUNT))


VAL_IMG_COUNT = tf.data.experimental.cardinality(val_list_ds).numpy()

print("Validating images count: " + str(VAL_IMG_COUNT))

Training images count: 4185
Validating images count: 1047


Train 데이터 셋, validation 데이터 셋 갯수 확인

In [9]:
CLASS_NAMES = np.array([str(tf.strings.split(item, os.path.sep)[-1].numpy())[2:-1]
                        for item in tf.io.gfile.glob(str(ROOT_PATH + "/chest_xray/train/*"))])
print(CLASS_NAMES)

['NORMAL' 'PNEUMONIA']


라벨 이름들을 한번 확인

현재 데이터에 라벨 데이터가 없기 때문에 라벨 데이터 만드는 함수 추가

In [15]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == "PNEUMONIA"   # 폐렴이면 양성(True), 노말이면 음성(False)를 리턴하게 합니다.

이미지 데이터는 사이즈가 제각각이기 때문에 이미지 사이즈를 맞춰주는 decode_img와 proecess_path 함수 생성함

In [12]:
def decode_img(img):
  # 이미지를 uint8 tensor로 바꾼다.
  img = tf.image.decode_jpeg(img, channels=3)
  # img를 범위 [0,1]의 float32 데이터 타입으로 바꾼다.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # img의 이미지 사이즈를 IMAGE_SIZE에서 지정한 사이즈로 수정한다.
  return tf.image.resize(img, IMAGE_SIZE)

def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [16]:
train_ds = train_list_ds.map(process_path, num_parallel_calls = AUTOTUNE)
val_ds = val_list_ds.map(process_path, num_parallel_calls = AUTOTUNE)

num_parallel_calls 파라미터에서 set-up에서 초기화 한 AUTOTUNE 이용시 빠른 데이터 처리 가능

In [20]:
for image, label in train_ds.take(4):
    print("image shape: ", image.numpy().shape)
    print("Label:", label.numpy())

image shape:  (180, 180, 3)
Label: True
image shape:  (180, 180, 3)
Label: True
image shape:  (180, 180, 3)
Label: True
image shape:  (180, 180, 3)
Label: True


작업이 잘 완료된것을 확인

test data에도 같은 작업을 진행함

In [21]:
test_list_ds =  tf.data.Dataset.list_files(str(ROOT_PATH + '/chest_xray/test/*/*'))
TEST_IMAGE_COUNT = tf.data.experimental.cardinality(test_list_ds).numpy()
test_ds = test_list_ds.map(process_path, num_parallel_calls = AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)

print(TEST_IMAGE_COUNT)

624


prepare_for_training() 함수를 통해 학습 데이터를 효율적으로 할 수 있도록 데이터를 변환 

* shuffle() 고정 크기 버퍼 유지, 해당 버퍼에서 무작위로 균일하게 다음 요소 선택

* repeat() epoch를 진행하면서 repeat을 통해 데이터세을 여러번 사용할수 있게 맞춰줌

* batch() BATCH_SIZE에서 정한만큼 배치로 지정

* prefetch() 학습 데이터를 나눠 읽기 때문에 데이터를 GPU에서 학습중에 다음 데ㅌ

 참고한 [캐글 노트북](https://www.kaggle.com/amyjang/tensorflow-pneumonia-classification-on-x-rays)